# Word2vec + deep learning trong dự đoán khuynh hướng stock

## Load data from google sheet 
https://docs.google.com/spreadsheets/d/1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o/edit#gid=732292548

In [1]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('./questioniso-0a60ce8c39c1.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

sheet = client.open_by_key('1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.worksheet('news')

data = sheet_instance.get_all_records()
print('data', data[:1])

data [{'id': 1, 'title': 'Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên giảm liên tiếp', 'content': 'Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\

In [2]:
df = pd.DataFrame()
title = []
content = []
date = []
stock_price = []
status = []

for values in data:
    title.append(values['title'])
    content.append(values['content'])
    date.append(values['date'])
    stock_price.append(values['stock_price'])
    status.append(values['status'])

df['title'] = title
df['content'] = content
df['date'] = date
df['stock_price'] = stock_price
df['status'] = status

df[0:5]

,title,content,date,stock_price,status
0,"Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên ...",Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3)...,,100.0,1.0
1,Vàng thế giới lùi nhẹ khi lợi suất trái phiếu ...,Giá vàng giảm vào ngày thứ Ba (14/3) khi lợi s...,,98.0,-0.5
2,Xét xử vụ DongABank: 'Lạnh gáy' với khoản vay ...,Sau khi bị cáo Phùng Ngọc Khánh (cựu Chủ tịch ...,,95.0,-1.0
3,Cựu tổng giám đốc Ngân hàng Đông Á Trần Phương...,Cùng các đồng phạm gây thiệt hại tổng số tiền ...,,93.0,-1.0
4,Nóng: Ngân hàng Nhà nước giảm lãi suất điều hành,Ngân hàng Nhà nước (NHNN) vừa ban hành các quy...,,94.0,1.0


In [3]:
import pandas as pd
from datetime import date, timedelta
import re
from underthesea import word_tokenize, sent_tokenize
from gensim import corpora, models

def clean_text(text):
    text = str(text).lower() # lowercase text
    return text

df = df[df['stock_price'] != '']

documents = df['content'].values.tolist()
print('documents:', documents[0:1])
print('len:', len(documents))

documents: ['Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\n\nChứng chỉ quỹ SPDR S&P Regional Banking ETF tiến 2%, phục hồi phần nào sau đà lao dốc

In [4]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

docs_tokened = []
documents_cleaned = []
for doc in documents:
    words = word_tokenize(str(doc).lower(), format="text").split(' ')
    words = [word.strip(".,?!") for word in words if word.isalpha()]
    docs_tokened.append(words)
    documents_cleaned.append(' '.join(words))

In [5]:
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

w2v_model = gensim.models.KeyedVectors.load_word2vec_format('../model/wiki.vi.model.bin', binary=True)

VISUAL this model

In [6]:
# Generate embeddings for each word
embedded_text = [[w2v_model[word] for word in words if word in w2v_model] for words in docs_tokened]
print(embedded_text[:2])

[[array([ 1.3178378e+00, -1.2601306e+00, -1.0148216e+00,  1.0622805e+00,
        9.5681000e-01,  9.2204779e-01,  3.9705551e-01, -1.0650949e+00,
       -7.0823359e-01,  4.3247962e-01,  1.0600468e+00, -4.4928390e-01,
        1.9078120e+00,  7.6190764e-01, -1.7369561e+00, -2.2159302e+00,
        1.7859336e+00, -2.9025981e-01,  8.1263518e-01, -1.5624757e-01,
       -7.9800999e-01,  1.8454561e-01, -2.0601952e+00,  4.3959242e-01,
        1.3735526e+00, -3.1910267e-01,  2.1093534e-01, -1.0014917e+00,
        1.3600535e+00,  2.7725545e-01, -8.6596560e-01,  4.8573557e-03,
       -1.3462207e+00,  8.6338675e-01, -1.7377094e+00, -2.9261301e+00,
       -5.0362879e-01,  2.5579423e-01,  1.3336146e+00,  6.1675072e-01,
        1.3726299e+00, -1.0479169e+00,  3.7747639e-01,  3.9230773e+00,
        1.4614642e+00,  1.3061544e+00,  1.1406986e+00, -7.8704911e-01,
        1.6861802e+00, -8.1537318e-01,  1.5631998e+00,  1.4272393e+00,
        9.7989964e-01, -7.8309709e-01, -1.6220077e+00,  1.5549201e-01,
    

In [8]:
statusNp = np.asarray(df['status'].values.tolist())
statusNp = np.expand_dims(statusNp, axis = 1)
print(statusNp.shape, statusNp)

(19, 1) [[ 1. ]
 [-0.5]
 [-1. ]
 [-1. ]
 [ 1. ]
 [-1. ]
 [-1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [-1. ]
 [ 0.9]
 [-1. ]
 [-1. ]
 [-1. ]
 [ 0.5]
 [ 0.1]
 [ 0.4]]


In [9]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing import sequence

tokenizer = Tokenizer()
tokenizer.fit_on_texts(documents)
sequences = tokenizer.texts_to_sequences(documents)

maxlen = 100
X = sequence.pad_sequences(sequences, maxlen=maxlen)
print('X:\n', X[:2])


X:
 [[   1   46   86   23   24  126  318  293   59   47   13  101  209  108
     8   59   62 1016 1017    8   28   77   73    4  294  146   55   95
   357  146  186  797  798    1  786  787   52   13    7   16    1  293
    10  153   48  574  172    8  147  269    1    4   22   55   95  293
  1018  799   40  270 1019   24  319  228    2  165  295   13  108    8
    59   54   17 1020  108    8   28  358    1    4   22   55   95   10
   101   43    5   61   52   13    7   16   10   43   43  574  172    8
    28   77]
 [ 225   54   32   61   22   15   29  402  675  122  194   96   45   93
   658   53    1    6    7  312  313  171   83  323   36   38  174 1026
  1027 1028  149  155  156  130   36   38   39 1029  184   49  676  579
   147  269  100  403  319  129  128  321   27  324  404   90   52   60
   272   56  175  176  170   10   62  677   78  670   54  148  186  460
     1 1030  325   36   38  326   87  210 1031   21  131  132    8   65
   461   12   78  138   58    2  176  170 1032 

In [36]:
import numpy as np

embedding_dim = 400
word_index = tokenizer.word_index
num_words = min(len(word_index) + 1, len(w2v_model.index_to_key))
embedding_matrix = np.zeros((num_words, embedding_dim))

print('num_words:', word_index)
for word, i in word_index.items():
    print('word, i:', word, i)
    if i >= num_words:
        continue
    if word in w2v_model.index_to_key:
        embedding_matrix[i] = w2v_model.word_vec(word)


num_words: {'của': 1, 'và': 2, 'cho': 3, 'các': 4, 'trong': 5, 'ngân': 6, 'hàng': 7, 'với': 8, 'công': 9, 'là': 10, 'có': 11, 'đồng': 12, 'tăng': 13, 'này': 14, 'đầu': 15, 'năm': 16, 'một': 17, 'tỷ': 18, 'lãi': 19, 'ty': 20, 'vào': 21, 'nhà': 22, 'số': 23, 'giá': 24, 'tiền': 25, 'chính': 26, 'sẽ': 27, 'dự': 28, 'tư': 29, 'không': 30, 'fed': 31, 'đó': 32, 'đã': 33, 'tài': 34, 'hơn': 35, 'thị': 36, 'được': 37, 'trường': 38, 'tại': 39, 'vốn': 40, 'vay': 41, 'suất': 42, '5': 43, 'ông': 44, 'sau': 45, 'mỹ': 46, '2': 47, 'cũng': 48, 'định': 49, 'bị': 50, 'phát': 51, 'mức': 52, 'sản': 53, 'trước': 54, 'kinh': 55, 'về': 56, '3': 57, 'giảm': 58, 'tháng': 59, 'quan': 60, 'khi': 61, '1': 62, 'để': 63, 'chủ': 64, 'điều': 65, 'ngày': 66, 'lại': 67, 'theo': 68, 'thể': 69, 'cơ': 70, 'cổ': 71, 'rằng': 72, 'từ': 73, 'phần': 74, 'kế': 75, 'ra': 76, 'báo': 77, 'usd': 78, 'động': 79, 'doanh': 80, 'điểm': 81, 'còn': 82, 'svb': 83, 'án': 84, 'việc': 85, 'chỉ': 86, 'liên': 87, 'ở': 88, 'hiện': 89, 'trên': 90

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_49363/673761574.py:14: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  embedding_matrix[i] = w2v_model.word_vec(word)


word, i: trịnh 922
word, i: khắc 923
word, i: tuyến 924
word, i: lựa 925
word, i: chọn 926
word, i: hạ 927
word, i: lưỡng 928
word, i: muốn 929
word, i: lắng 930
word, i: sớm 931
word, i: ethan 932
word, i: thiểu 933
word, i: jamie 934
word, i: đô 935
word, i: sẵn 936
word, i: sàng 937
word, i: citigroup 938
word, i: cảm 939
word, i: hứa 940
word, i: mời 941
word, i: dấy 942
word, i: todd 943
word, i: hút 944
word, i: chắc 945
word, i: thụy 946
word, i: credit 947
word, i: suisse 948
word, i: mong 949
word, i: thảo 950
word, i: trễ 951
word, i: thật 952
word, i: kêu 953
word, i: khía 954
word, i: tuyên 955
word, i: sắp 956
word, i: đọc 957
word, i: iii 958
word, i: nửa 959
word, i: bankingbook 960
word, i: 200 961
word, i: repricing 962
word, i: gap 963
word, i: htm 964
word, i: đáo 965
word, i: phụ 966
word, i: nhập 967
word, i: tỉnh 968
word, i: thự 969
word, i: tầng 970
word, i: trí 971
word, i: sông 972
word, i: ranh 973
word, i: nghỉ 974
word, i: 5ha 975
word, i: thuê 976
word, i:

In [32]:
print('word_index:', len(word_index))
print(embedding_matrix.shape)
print(embedding_matrix[0:20])

word_index: 1511
(1512, 400)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.93096751  0.46870598 -1.82646453 ... -2.17166424  0.92262614
   0.00371891]
 [ 1.51843655  0.70282888 -1.45200372 ... -0.75952691  0.73407036
   0.21997593]
 ...
 [ 0.97361964 -0.31454906 -1.03614855 ... -0.36377674  0.95031494
  -0.14124921]
 [ 1.21238697 -1.7832644  -2.31296849 ... -0.7070995   1.67624152
   2.18489075]
 [-0.8703934   1.30716443 -0.85750818 ... -2.07618403  0.05252319
   1.39935327]]


In [52]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(
        100, 
        return_sequences = True, 
        recurrent_dropout=0.2)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 100, 400)          604800    
                                                                 
 bidirectional_2 (Bidirectio  (None, 100, 200)         400800    
 nal)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 20000)             0         
                                                                 
 dense_7 (Dense)             (None, 1)                 20001     
                                                                 
Total params: 1,025,601
Trainable params: 420,801
Non-trainable params: 604,800
_________________________________________________________________


In [53]:
model.fit(X, statusNp, epochs=30, batch_size=32)

Epoch 1/30
1/1 [==============================] - 3s 3s/step - loss: 0.8370 - acc: 0.1579
Epoch 2/30
1/1 [==============================] - 0s 195ms/step - loss: -1.0455 - acc: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 169ms/step - loss: -2.4976 - acc: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 0s 208ms/step - loss: -4.0415 - acc: 0.0000e+00
Epoch 5/30
1/1 [==============================] - 0s 163ms/step - loss: -5.7376 - acc: 0.0000e+00
Epoch 6/30
1/1 [==============================] - 0s 184ms/step - loss: -7.6281 - acc: 0.0000e+00
Epoch 7/30
1/1 [==============================] - 0s 182ms/step - loss: -9.7838 - acc: 0.0526
Epoch 8/30
1/1 [==============================] - 0s 194ms/step - loss: -12.2686 - acc: 0.0526
Epoch 9/30
1/1 [==============================] - 0s 205ms/step - loss: -15.1093 - acc: 0.0526
Epoch 10/30
1/1 [==============================] - 0s 174ms/step - loss: -18.3382 - acc: 0.1053
Epoch 11/30
1/1 [=======================

In [54]:
test_data = [
    """
    Công ty Lọc hóa dầu Bình Sơn (BSR) lên kế hoạch bảo dưỡng lớn cho nhà máy lọc dầu Dung Quất, đề ra mức lãi giảm gần 90%.
    """
]

sequences_test = tokenizer.texts_to_sequences(test_data)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)
print('X:\n', X_test[:2])

result = model.predict(X_test)
print('result:\n', result)

X:
 [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    9   20  422  338  203  105 1079  283  121   75  102   96
   423  109    3   22  373  422  203  595  596  219   76   52   19   58
   145  576]]
1/1 [==============================] - 0s 248ms/step
result:
 [[0.]]
